In this notebook we approach the Titanic dataset with a simple logistic regression. 

In [38]:
import numpy as np
import pandas as pd
from math import log
%matplotlib inline 
import matplotlib.pyplot as plt
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder
warnings.filterwarnings('ignore')


train_data = pd.read_csv("Titanic/train.csv")
test_data = pd.read_csv("Titanic/test.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
train_data = train_data.copy()
test_data = test_data.copy()
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [40]:
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [41]:
for col in train_data.columns:
    if train_data[col].isna().any():
        print(train_data[col].isna().value_counts())

Age
False    714
True     177
Name: count, dtype: int64
Cabin
True     687
False    204
Name: count, dtype: int64
Embarked
False    889
True       2
Name: count, dtype: int64


From the above, it seems likely that name, cabin and ticket won't  be useful features. Clearly one can't predict whether someone survived by their name (unless, maybe, it starts with 'Lord', or something like this). Cabin is missing 3/4 of its features, and even if it weren't, there would be too many different entries to one hot encode. One could group cabins in some way, but this would, I believe at best, become a proxy for class. Similarly, using the ticket number as a predictive feature would be hard (at least, I'm not sure what to do with it). 

In the solution to this problem as an excercise in Hands On Machine Learning, the author suggests that one should group SibSp and Parch together as a feature Family_on_board. 

In [42]:
features = ['Survived', 'Pclass', 'Sex', 'Fare', 'Embarked', 'FamilyOnBoard', 'Age']
train_data['FamilyOnBoard'] = train_data['Parch'] + train_data['SibSp']
test_data['FamilyOnBoard'] = test_data['Parch'] + test_data['SibSp']
train_data_features = train_data[features]
train_data_features.groupby('Pclass',as_index = False)['Age'].describe()

,Pclass,count,mean,std,min,25%,50%,75%,max
0,1,186.0,38.233441,14.802856,0.92,27.0,37.0,49.0,80.0
1,2,173.0,29.877630,14.001077,0.67,23.0,29.0,36.0,70.0
2,3,355.0,25.140620,12.495398,0.42,18.0,24.0,32.0,74.0


So there seems to be a difference in average age between the classes, but I will just impute any missing values as the average. It could be a potential improvement to impute different ages for different classes.

In [43]:
y = train_data_features['Survived']
X = train_data_features.drop('Survived', axis = 1)
X['FamilyOnBoard'].value_counts()

FamilyOnBoard
0     537
1     161
2     102
3      29
5      22
4      15
6      12
10      7
7       6
Name: count, dtype: int64

In [44]:
OHE_encoder = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy='most_frequent')),
        ("encoder", OneHotEncoder())
    ]
)
num_pipeline = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy = 'mean')),
        ("scaler", StandardScaler())
    ]
)
preprocessor = ColumnTransformer(transformers = [
    ("num", num_pipeline, ['Fare', 'Age', 'FamilyOnBoard']), 
    ("ord", OrdinalEncoder(), ['Sex']),
    ("OHE", OHE_encoder, ['Embarked'])
])


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)
model = LogisticRegression()
my_pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                              ('model', model)
                             ])
my_pipeline.fit(X_train,y_train)
y_pred = my_pipeline.predict(X_train)
Acc_score = accuracy_score(y_train, y_pred)
print(Acc_score)
print(confusion_matrix(y_train, y_pred))

0.7935393258426966
[[386  58]
 [ 89 179]]


In [46]:
y_pred_test = my_pipeline.predict(X_test)
Acc_score = accuracy_score(y_test, y_pred_test)
print(Acc_score)
print(confusion_matrix(y_test, y_pred_test))

0.7877094972067039
[[90 15]
 [23 51]]


In [47]:
predictions = my_pipeline.predict(test_data)
my_submission = pd.DataFrame({'PassengerId': test_data.index, 'Survived': predictions})
my_submission.to_csv('RefinedLogitSubmission.csv', index=False)

This results in a score of .76555, which isn't any better than what I got with less feature engineering. It seems like the model has been changed, but not by enough to push some of the predictions over the threshold. I could compare the two models with binary cross entropy to see whether the predictions are going in the correct direction. Other improvements include:
 * I could create a custom test set which was designed to be representative of the data. Looking at X_test.describe(), it does seem like we could make a better train-test split, but I'm not sure how much of a difference this would make.
 * Given that the only about 38% of passengers survived, I wonder if it might make sense to increase the threshold for predicting survive slightly? Looking at the confusion matrix, this might helt -- the number of false negatives (in this case, the number of incorrectly predicted survivals) is higher than the number of false positives. 